# Object Detection using the YOLO V4 pre-trained model

*by Georgios K. Ouzounis, June 10th, 2021*

In this exercise we will experiment with object detection in still images using the YOLO V4 pretrained model 

## Setup

In [ ]:
# import the relevant libraries
import numpy as np
import cv2 # openCV

In [ ]:
# check the opencv version
print(cv2.__version__)

In [ ]:
# if the openCV version is < 4.4.0 update to the latest otherwise skip this step
!pip install opencv-python==4.5.2.52

## Get the model



In [ ]:
# first create a directory to store the model
%mkdir model

In [ ]:
# enter the directory and download the necessary files 
%cd model
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/coco.names
%cd ..

##  Get the test data

In [ ]:
# option 1: download a test image from the web
!wget https://github.com/georgiosouzounis/object-detection-yolov4/raw/main/data/pretrained/people_bicycles.jpg

In [ ]:
#option 2: mount your Google Drive and select the image of your own
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# copy a test file locally or use it directly from the mounted drive 
# customize this example for your drive structure
%cp drive/MyDrive/object_detection/people_bicycles.jpg .

## Read test image

In [ ]:
# read file
test_img = cv2.imread('people_bicycles.jpg')

In [ ]:
# display test image
# import the cv2_imshow as a replacement of cv2.imshow to prevent errors
from google.colab.patches import cv2_imshow
cv2_imshow(test_img)

## Image2Blob

convert the image to blob for model compatibility using the opencv builtin  dnn.blobFromImage() method

Argument explanations:
- scalefactor: multiplication factor for each pixel to rescale its intensity in  the range of [0,1]. No contrast stretching is performed. It is set to 1/255.0 = 0.003922.
- new_size: rescaling size for network compatibility. We use (416, 416). Other supported sizes are (320, 320) and (609, 609). The greater the size is the better the prediction accuracy will be but at the cost of higher computational load.
- swapRB: binary flag that if set instructs opencv to swap the red and blue channels. That is because opencv stores images in a BGR order but YOLO requires them in RGB.
- crop: binary flag that if set instructs opencv to crop the image after resizing.


In [ ]:
scalefactor = 1.0/255.0
new_size = (416, 416)
blob = cv2.dnn.blobFromImage(test_img, scalefactor, new_size, swapRB=True, crop=False)

## Customize the YOLO detector

class labels:

In [ ]:
class_labels_path = "/content/model/coco.names"
class_labels = open(class_labels_path).read().strip().split("\n")
class_labels

bounding box color definitions; two options:

1. create a template set of colors that is repeated till each class gets one assigned

In [ ]:
# declare repeating bounding box colors for each class 
# 1st: create a list colors as an RGB string array
# Example: Red, Green, Blue, Yellow, Magenda
class_colors = ["255,0,0","0,255,0","0,0,255","255,255,0","255,0, 255"]

#2nd: split the array on comma-seperated strings and for change each string type to integer
class_colors = [np.array(every_color.split(",")).astype("int") for every_color in class_colors]

#3d: convert the array or arrays to a numpy array
class_colors = np.array(class_colors)

#4th: tile this to get 80 class colors, i.e. as many as the classes  (16rows of 5cols each). 
# If you want unique colors for each class you may randomize the color generation 
# or set them manually
class_colors = np.tile(class_colors,(16,1))

to visualize the effect of this code execute the next block

In [ ]:
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)

for i in range(16):
  line = ""
  for j in range(5):
    class_id = i*5 + j
    class_id_str = str(class_id)
    text = "class" + class_id_str
    colored_text = colored(class_colors[class_id][0], class_colors[class_id][1], class_colors[class_id][2], text)
    line += colored_text
  print(line)

2. or simply create random colors:

In [ ]:
class_colors = np.random.randint(0, 255, size=(len(class_labels), 3), dtype="uint8")

## Load and run the model

In [ ]:
# Load the pre-trained model 
yolo_model = cv2.dnn.readNetFromDarknet('model/yolov4.cfg','model/yolov4.weights')

In [ ]:
# Read the network layers/components. The YOLO V4 neural network has 379 components.
# They consist of convolutional layers (conv), rectifier linear units (relu) etc.:
model_layers = yolo_model.getLayerNames()
print("number of network components: " + str(len(model_layers))) 
print(model_layers)

In [ ]:
# extract the output layers

# in the code that follows:
# - model_layer[0]: returns the index of each output layer in the range of 1 to 379
# - model_layer[0] - 1: corrects  this to the range of 0 to 378
# - model_layers[model_layer[0] - 1]: returns the indexed layer name 
output_layers = [model_layers[model_layer[0] - 1] for model_layer in yolo_model.getUnconnectedOutLayers()]

# YOLOv4 deploys the same YOLO head as YOLOv3 for detection with   
# the anchor based detection steps, and three levels of detection granularity. 
print(output_layers)

In [ ]:
# input pre-processed blob into the model
yolo_model.setInput(blob)

# compute the forward pass for the input, storing the results per output layer in a list
obj_detections_in_layers = yolo_model.forward(output_layers)

# verify the number of sets of detections
print("number of sets of detections: " + str(len(obj_detections_in_layers)))

Note: the yolo architecture offers 3 detection layers that generate feature maps of sizes 15x15x255, 30x30x255 and 60x60x255 respectively. Each detection layer is tasked with finding objects of a given size range.

<img src="https://blog.roboflow.com/content/images/2020/06/image-19.png" width="300"/>

## Analyze the results

The objective now is to get each object detection from each output layer and evaluate  the algorithm's cofidence score against a threshold. For high confidence detections we extract the class ID and the bounding box info.

In [ ]:
def object_detection_analysis(test_image, obj_detections_in_layers, confidence_threshold): 

  # get the image dimensions  
  img_height = test_img.shape[0]
  img_width = test_img.shape[1]

  result = test_image.copy()
  
  # loop over each output layer 
  for object_detections_in_single_layer in obj_detections_in_layers:
	  # loop over the detections in each layer
      for object_detection in object_detections_in_single_layer:  
        # obj_detection[1]: bbox center pt_x
        # obj_detection[2]: bbox center pt_y
        # obj_detection[3]: bbox width
        # obj_detection[4]: bbox height
        # obj_detection[5]: confidence scores for all detections within the bbox 

        # get the confidence scores of all objects detected with the bounding box
        prediction_scores = object_detection[5:]
        # consider the highest score being associated with the winning class
        # get the class ID from the index of the highest score 
        predicted_class_id = np.argmax(prediction_scores)
        # get the prediction confidence
        prediction_confidence = prediction_scores[predicted_class_id]
    
        # consider object detections with confidence score higher than threshold
        if prediction_confidence > confidence_threshold:
            # get the predicted label
            predicted_class_label = class_labels[predicted_class_id]
            # compute the bounding box cooridnates scaled for the input image 
            # scaling is a multiplication of the float coordinate with the appropriate  image dimension
            bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
            # get the bounding box centroid (x,y), width and height as integers
            (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
            # to get the start x and y coordinates we to subtract from the centroid half the width and half the height respectively 
            # for even values of width and height of bboxes adjacent to the  image border
            #  this may genetrate a -1 which is prevented by the max() operator below  
            start_x_pt = max(0, int(box_center_x_pt - (box_width / 2)))
            start_y_pt = max(0, int(box_center_y_pt - (box_height / 2)))
            end_x_pt = start_x_pt + box_width
            end_y_pt = start_y_pt + box_height
            
            # get a random mask color from the numpy array of colors
            box_color = class_colors[predicted_class_id]
            
            # convert the color numpy array as a list and apply to text and box
            box_color = [int(c) for c in box_color]
            
            # print the prediction in console
            predicted_class_label = "{}: {:.2f}%".format(predicted_class_label, prediction_confidence * 100)
            print("predicted object {}".format(predicted_class_label))
            
            # draw the rectangle and text in the image
            cv2.rectangle(result, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), box_color, 1)
            cv2.putText(result, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)
  return result


In [ ]:
confidence_threshold = 0.2
result_raw = object_detection_analysis(test_img, obj_detections_in_layers, confidence_threshold)

## Visualize the detections

In [ ]:
cv2_imshow(result_raw)

## Non Maxima Suppression (NMS)

supress fully- or partially- overlapping bounding boxes

In [ ]:
# step 1: declare lists for the arguments of interest: classID, bbox info, detection confidences
class_ids_list = []
boxes_list = []
confidences_list = []

The next 4 blocks of code area available as a single function in ```object_detections_functions.py```

In [ ]:
# step 2: populate those lists from the detections

def object_detection_attributes(test_image, obj_detections_in_layers, confidence_threshold):
  # get the image dimensions  
  img_height = test_img.shape[0]
  img_width = test_img.shape[1]
  
  # loop over each output layer 
  for object_detections_in_single_layer in obj_detections_in_layers:
    # loop over the detections in each layer
    for object_detection in object_detections_in_single_layer:  
      # get the confidence scores of all objects detected with the bounding box
      prediction_scores = object_detection[5:]
      # consider the highest score being associated with the winning class
      # get the class ID from the index of the highest score 
      predicted_class_id = np.argmax(prediction_scores)
      # get the prediction confidence
      prediction_confidence = prediction_scores[predicted_class_id]
      
      # consider object detections with confidence score higher than threshold
      if prediction_confidence > confidence_threshold:
        # get the predicted label
        predicted_class_label = class_labels[predicted_class_id]
        # compute the bounding box cooridnates scaled for the input image
        bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
        (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
        start_x_pt = max(0, int(box_center_x_pt - (box_width / 2)))
        start_y_pt = max(0, int(box_center_y_pt - (box_height / 2)))
        
        # update the 3 lists for nms processing
        # - confidence is needed as a float 
        # - the bbox info has the openCV Rect format
        class_ids_list.append(predicted_class_id)
        confidences_list.append(float(prediction_confidence))
        boxes_list.append([int(start_x_pt), int(start_y_pt), int(box_width), int(box_height)])

populate the lists with our detections

In [ ]:
score_threshold = 0.5
object_detection_attributes(test_img, obj_detections_in_layers, score_threshold)

compute the [NMS](https://docs.opencv.org/master/d6/d0f/group__dnn.html#ga9d118d70a1659af729d01b10233213ee)


In [ ]:
# NMS for a set of overlapping bboxes returns the ID of the one with highest 
# confidence score while suppressing all others (non maxima)
# - score_threshold: a threshold used to filter boxes by score 
# - nms_threshold: a threshold used in non maximum suppression. 

score_threshold = 0.5
nms_threshold = 0.4
winner_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, score_threshold, nms_threshold)

In [ ]:
# loop through the final set of detections remaining after NMS and draw bounding box and write text
for winner_id in winner_ids:
    max_class_id = winner_id[0]
    box = boxes_list[max_class_id]
    start_x_pt = box[0]
    start_y_pt = box[1]
    box_width = box[2]
    box_height = box[3]
    
    #get the predicted class id and label
    predicted_class_id = class_ids_list[max_class_id]
    predicted_class_label = class_labels[predicted_class_id]
    prediction_confidence = confidences_list[max_class_id]

    #obtain the bounding box end co-oridnates
    end_x_pt = start_x_pt + box_width
    end_y_pt = start_y_pt + box_height
    
    #get a random mask color from the numpy array of colors
    box_color = class_colors[predicted_class_id]
    
    #convert the color numpy array as a list and apply to text and box
    box_color = [int(c) for c in box_color]
    
    # print the prediction in console
    predicted_class_label = "{}: {:.2f}%".format(predicted_class_label, prediction_confidence * 100)
    print("predicted object {}".format(predicted_class_label))
    
    # draw rectangle and text in the image
    cv2.rectangle(test_img, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), box_color, 1)
    cv2.putText(test_img, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)

In [ ]:
cv2_imshow(test_img)